In [1]:
import os
%pwd

'c:\\Users\\lenovo\\Desktop\\Stage\\text-to-3D_Model_Generation\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\lenovo\\Desktop\\Stage\\text-to-3D_Model_Generation'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataRenderingConfig:
    local_data_file : Path
    render_script : Path
    output_dir : Path
    num_images : int
    resolution : int
    engine : str
    camera_dist: float

In [5]:
from textTo3DModelGen.constants import *
from textTo3DModelGen.utils.common import read_yaml, create_directories

c:\Users\lenovo\Desktop\Stage\text-to-3D_Model_Generation\.venv37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = HYPER_PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([
            self.config.artifacts_root
        ])

    def get_data_rendering_config(self) -> DataRenderingConfig:
        config = self.config.data_rendering

        create_directories([
            config.output_dir
        ])

        data_rendering_config = DataRenderingConfig(
            local_data_file = config.local_data_file,
            render_script = config.render_script,
            output_dir = config.output_dir,
            num_images = config.num_images,
            resolution = config.resolution,
            engine = config.engine,
            camera_dist = config.camera_dist
        )

        return data_rendering_config


In [7]:
from textTo3DModelGen import logger
from textTo3DModelGen.utils.common import get_size
import pandas as pd

In [8]:
class DataRendering:
    def __init__(self, config: DataRenderingConfig):
        self.config = config
        self.render_cmd = f"blender -b -P {self.config.render_script} -- \
                            --object_path %s \
                            --output_dir {self.config.output_dir} \
                            --engine {self.config.engine} \
                            --num_images {self.config.num_images} \
                            --camera_dist {self.config.camera_dist} \
                            --resolution {self.config.resolution}"

    def render_all(self):
        try:
            data = pd.read_csv(self.config.local_data_file)
            logger.info(f"Read of data from {self.config.local_data_file} is done.")

            for obj in data["saved_path"]:
                file_path = obj.replace("/root", "C:/Users/lenovo")
                if os.path.exists(file_path):
                    logger.info(f"Start rendering of the model {os.path.basename(file_path)}.")
                    os.system(self.render_cmd%(file_path))
                    logger.info(f"End rendering of the model {os.path.basename(file_path)}.")
                else:
                    logger.info(f"The file {file_path} does not exist.")
        except Exception as e:
            raise e

In [10]:
try:
    config = ConfigurationManager()
    data_rendering_config = config.get_data_rendering_config()
    data_rendering = DataRendering(config= data_rendering_config)
    data_rendering.render_all()
except Exception as e:
    raise e

[2024-09-01 14:20:30,276: INFO: yaml file: config\config.yaml loaded successfully]
[2024-09-01 14:20:30,281: INFO: yaml file: hyper_params.yaml loaded successfully]
[2024-09-01 14:20:30,283: INFO: created directory at: artifacts]
[2024-09-01 14:20:30,285: INFO: created directory at: artifacts/dataset]
[2024-09-01 14:20:30,290: INFO: Read of data from artifacts/data_ingestion/objaverse_with_description.csv is done.]
[2024-09-01 14:20:30,295: INFO: Start rendering of the model ed51a51909ee46c780db3a85e821feb2.glb.]


[2024-09-01 14:20:56,135: INFO: End rendering of the model ed51a51909ee46c780db3a85e821feb2.glb.]
[2024-09-01 14:20:56,137: INFO: Start rendering of the model 9110b606f6c547b2980fcb3c8c4b6a1c.glb.]
[2024-09-01 14:23:29,085: INFO: End rendering of the model 9110b606f6c547b2980fcb3c8c4b6a1c.glb.]
